# Analyzing CIA Factbook Data Using SQL

First, We'll use the following code to connect our Jupyter Notebook to our database file:

In [3]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Overview of the data

Let's run a test to see if the database is working and get a sense of what the data looks like.

In [2]:
%%sql
SELECT *
FROM facts 
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The database is working correctly as expected.

Here are the descriptions for some of the columns:

- name - The name of the country.
- area - The total land and sea area of the country.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.

Let's start by calculating some summary statistics and see what they tell us.

## Summary Statiscts

Let's run a query that returns the following:

- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth

In [13]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Both minimum population and maximum population don't make sense (0 and more than 7 billion, respectively). Let's run two queries to find out what countries these are.

In [22]:
%%sql
SELECT name
FROM facts
WHERE population = (SELECT MIN(population) FROM facts)

 * sqlite:///factbook.db
Done.


name
Antarctica


In [23]:
%%sql
SELECT name
FROM facts
WHERE population = (SELECT MAX(population) FROM facts)

 * sqlite:///factbook.db
Done.


name
World


According to the CIA factbook, there's nobody living in Antarctica. Besides that, the country with more than 7 billion people is not actually a country, but the whole world. Let's recalculate the summary statics, but this time excluding the the row for the whole world.

In [25]:
%%sql
SELECT MIN(population) AS min_pop, 
       MAX(population) AS max_pop, 
       MIN(population_growth) AS min_pop_growth, 
       MAX(population_growth) AS max_pop_growth
FROM facts
WHERE population != (SELECT MAX(population) FROM facts)

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


Also, let's calculate the average value for the following columns:

- population 
- area

In [27]:
%%sql
SELECT AVG(population) AS avg_pop,
       AVG(area) AS avg_area
FROM facts
WHERE name <> 'World'

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


We see that the average population is around 32 million and the average area is 555 thousand square kilometers.

Next, let's write a query to finds all countries meeting both of the following criteria:

- the population is above average.
- the area is below average.


In [35]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population) 
                    FROM facts 
                    WHERE name <> 'World')
      AND
     area < (SELECT AVG(area)
             FROM facts
             WHERE name <> 'World')
        
    

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


The list of the countries above are densily populated (big populations living in areas below the average).

# Next steps

### Which country has the most people? 

In [37]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MAX(population)
                    FROM facts
                    WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


China is the most populated country in the world with approximatelly 1.4 billion people. The TOP 5 countries in population are in the table below.

In [47]:
%%sql
SELECT *
FROM facts
WHERE name <> 'World' AND name <> 'European Union'
ORDER BY population DESC
LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14


### Which country has the highest growth rate?

In [39]:
%%sql
SELECT *
FROM facts
WHERE population_growth = (SELECT MAX(population_growth)
                    FROM facts
                    WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


South Sudan has the highest population growth rate: 4.02. Other countries that have high population growth rates can be seen in the table below.

In [41]:
%%sql
SELECT *
FROM facts
WHERE name <> 'World'
ORDER BY population_growth DESC
LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74


### Which countries have the highest ratios of water to land?

In [61]:
%%sql
SELECT *, ROUND(CAST(area_water AS float)/CAST(area_land AS float), 4) as ratio_water_land
FROM facts
WHERE name <> 'World' 
      AND name <> 'European Union' 
      AND area_land != 'None'
      AND area_water != 'None'
ORDER BY ratio_water_land DESC
LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,ratio_water_land
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,905.6667
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,4.5202
246,rq,Puerto Rico,13791,8870,4921,3598357,0.6,10.86,8.67,8.15,0.5548
12,bf,"Bahamas, The",13880,10010,3870,324597,0.85,15.5,7.05,0.0,0.3866
71,pu,Guinea-Bissau,36125,28120,8005,1726170,1.91,33.38,14.33,0.0,0.2847
